In [11]:
import numpy as np
import pandas as pd
import multiprocessing as mp

#Definimos las constantes del programa
N = 1000       #Numero de elementos en los arreglos
CHUNK = 100    #Tamaño de las partes en las que dividiremos el trabajo
MOSTRAR = 10   #Cantidad de valores a imprimir

def llenar_arreglo(): #Funcion para llenar un arreglo con los numeros aleatorios entre 0 y 100
    return np.random.randint(0, 101, N).astype(float)


def suma_arreglos(a, b):     #Funcion para sumar los arreglos en paralelo
    c = np.zeros(N, dtype=float)  #Se crea un arreglo sin nada para la suma

    # Se determinan los procesos podemos usar
    num_procesos = min(8, mp.cpu_count())  #Max. 8 procesos o la cantidad de nucleos disponibles
    chunk_size = N // num_procesos  # Se divide el trabajo en partes iguales


    def suma_parcial(inicio, fin, resultado): #Funcion interna que cada proceso usara para calcular su parte de la suma
        for i in range(inicio, fin):
            resultado[i] = a[i] + b[i]  #Se suman los valores de los arreglos


    resultado = mp.Array('d', N)  #Se crea la estructura de memoria compartida para que todos los procesos puedan escribir en el mismo arreglo


    procesos = [] # Aqui se guardaran los procesos que vamos a crear
    for i in range(num_procesos):
        inicio = i * chunk_size  #Indice inicial para este proceso
        fin = N if i == num_procesos - 1 else (i + 1) * chunk_size  #Indice final

        p = mp.Process(target=suma_parcial, args=(inicio, fin, resultado)) #Creacion de un nuevo proceso que ejecuta la funcion de suma_parcial en su rango correspondiente
        procesos.append(p)
        p.start()  # El proceso se comienza

    for p in procesos:     #Se esoera a que todos los procesos terminen
        p.join()

    c[:] = np.array(resultado[:])     #Se convierten a el resultado compartido en un arreglo de numpy

    return c

#Se generan los arreglos con valores aleatorios
a = llenar_arreglo()
b = llenar_arreglo()

#Se hace la suma en paralelo
c = suma_arreglos(a, b)

#Se guardan los primeros valores en un DF para visualizar los resultados
df = pd.DataFrame({
    "Arreglo A": a[:MOSTRAR],
    "Arreglo B ": b[:MOSTRAR],
    "Arreglo C (Suma)": c[:MOSTRAR]  #suma de A + B
})

# Se imprimen los resultados
print(df)

   Arreglo A  Arreglo B   Arreglo C (Suma)
0       33.0        95.0             128.0
1       41.0        97.0             138.0
2       93.0        77.0             170.0
3       45.0         5.0              50.0
4       22.0        93.0             115.0
5       83.0        95.0             178.0
6       89.0        58.0             147.0
7       70.0        42.0             112.0
8       67.0        96.0             163.0
9        0.0        93.0              93.0
